In [1]:
import cv2
import numpy as np
import imutils
from skimage.metrics import structural_similarity as ssim

import tkinter as tk
from tkinter import font

from pathlib import Path
import pandas as pd

from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageOps
import matplotlib.pyplot as plt
import matplotlib
import os 
from ctypes import *
import re
import pathlib

In [2]:
# NOISE REDUCTION
def noise_reduction(image) :
    im = cv2.imread(image)
    yuv = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb) # or COLOR_BGR2YUV, should work equivalently here
    mean = np.mean(yuv, axis=(0,1))
    std = np.std(yuv, axis=(0,1))
    # EDIT: there needs to be an abs() on that difference, we want magnitudes, no signs
    mask = (np.abs(yuv - mean) / std >= 4.5).any(axis=2)
    mask_u8 = mask.astype(np.uint8) * 255

    print("noise_reduction for one image done!")
    return mask_u8

In [3]:
def find_defects(imB, imA):
    imageB = cv2.imread(imB)
    imageA = cv2.imread(imA)

    # convert the images to grayscale
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    (score, diff) = ssim(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")

    # threshold the difference image, followed by finding contours to
    # obtain the regions of the two input images that differ
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    # find largest area contour
    max_area = -1
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if area>max_area:
            cnt = contours[i]
            max_area = area

    # image = cv2.rectangle(image, start_point, end_point, color, thickness)
    # loop over the contours
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        if w < 550 and h < 735:
            cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2) #original
            cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2) #modified
            i +=1
    return imageB

In [ ]:
cad = ['out1.png', 'out2.png', 'out3.png', 'out4.png', 'out5.png', 'out6.png', 'out7.png', 'out8.png', 'out9.png', 'out10.png', 
        'out11.png', 'out12.png', 'out13.png', 'out14.png', 'out15.png', 'out16.png', 'out17.png', 'out18.png', 'out19.png', 'out20.png', 
        'out21.png', 'out22.png', 'out23.png', 'out24.png', 'out25.png', 'out26.png', 'out27.png', 'out28.png', 'out29.png', 'out30.png', 
        'out31.png', 'out32.png', 'out33.png', 'out34.png', 'out35.png', 'out36.png', 'out37.png', 'out38.png', 'out39.png', 'out40.png', 
        'out41.png', 'out42.png', 'out43.png', 'out44.png', 'out45.png', 'out46.png', 'out47.png', 'out48.png', 'out49.png', 'out50.png']

imageNum = 0


if not os.path.exists('tmp'):
    os.makedirs('tmp')

def set_plot_title(title, fs = 16):
    plt.title(title, fontsize = fs)


class MainApp:
    def __init__(self):
        def press():
            df1 = pd.read_excel('output_scores.xls', sheet_name='outputs')
            ii1 = df1['image number'].tolist()
            tt1 = df1['total time'].tolist()
            ff1 = df1['f1 score'].tolist()
            aa1 = df1['accuracy'].tolist()
            i1,t1,f1,a1 = [],[],[],[]

            for i in range(0,len(ii1)):
                i1.append([ii1[i]])
                t1.append([tt1[i]])
                f1.append([ff1[i]])
                a1.append([aa1[i]])

            if isinstance(imageNum, int):
                print("intt")
                im = int(imageNum)-1
            else :
                im = int(imageNum[0])-1

            print("imagenumm " , im)

            self.info_lbl.config(text=str("Image " + str(i1[im]) +  "\n\tTime\t: " + str(t1[im]) + "  second" + "\nF1 score\t: " + str(f1[im]) + "\nAccuracy\t: " + str(a1[im])))

        #Seeting up root
        windll.shcore.SetProcessDpiAwareness(1)
        #self.root = tk.Tk()
        self.root = tk.Toplevel()

        self.root.tk.call('tk', 'scaling', 1.5) # To get bigger window on higher resolution displays
        self.root.resizable(0, 0)
        self.root.title("Demo")
        #self.root.iconphoto(False, tk.PhotoImage(file = pathlib.Path("gui/icon.png")))

        #setting up left side of GUI
        self.left_frame = tk.LabelFrame(text = "Original Deflectometry Image")
        self.left_frame.columnconfigure(0)
        self.left_frame.columnconfigure(1)
        self.left_frame.rowconfigure(0)
        self.left_frame.rowconfigure(1)
        self.left_frame.rowconfigure(2)
        self.left_frame.rowconfigure(3)
        self.left_frame.rowconfigure(4)
        self.left_frame.rowconfigure(5)
        self.original_img = tk.Label(self.left_frame, image = "", text = "Load an image \n to preview it here!", padx = 350, pady = 300)
        buttonFont = font.Font(family='Helvetica', size=18, weight='bold')
        self.btn_browse_img = tk.Button(self.left_frame, text = "Browse Image", bg = "plum4", command = self.browse_img, font=buttonFont)# command ile functiona gidiyo
        self.btn_apply_filter = tk.Button(self.left_frame, text = "Find Defects", bg = "plum4", command = self.apply_filter, font=buttonFont)
        self.select_filter_var = tk.StringVar(self.root)
        self.select_filter_var.set('Ideal')
        self.frequency = tk.Entry(self.left_frame)
        self.original_img.grid(row = 0, column = 0, columnspan = 2)
        self.btn_browse_img.grid(row = 1, column = 0, sticky='nsew') # image ı arayüze koyuyor
        self.btn_apply_filter.grid(row = 1, column = 1, sticky='nsew')
        self.left_frame.pack(side = "left", fill = tk.Y)

        #setting up Right side of GUI
        self.right_frame = tk.LabelFrame(text = "Defect Detection")
        self.filter_img = tk.Label(self.right_frame, image = "", text = "Press Find Defects Button \nto view it here!", padx = 350, pady = 300)
        labelFont = font.Font(family='Helvetica', size=14)
    
        self.btn_summary = tk.Button(self.right_frame, text = "Show Details",  bg = "plum4", command = press, font=buttonFont)

        self.info_lbl = tk.Label(self.right_frame, text = "Press show details to performans results !", font=labelFont)
        
        self.filter_img.pack()
        self.btn_summary.pack(in_ = self.right_frame, fill = tk.X)
        self.info_lbl.pack(in_ = self.right_frame, fill = tk.BOTH, side = tk.LEFT)
        self.right_frame.pack(side = "right", fill = tk.Y) 

    def details():
        return "F1 score : " + "Time : "

    def browse_img(self):
        try:
            file = filedialog.askopenfilename(title = "Load Image", filetypes=[('Images', ['*jpeg','*png','*jpg', '*tif'])])
            temp = str(file)
            parse = temp.split('/')
            last = parse[-1]
            #print(parse[-1])
            global imageNum
            imageNum = re.findall(r'[0-9]+', last)
            #print(imageNum)

            file = Image.open(file)#

            file.save(pathlib.Path("tmp/original_img.png"))

            file = file.resize((864,648), Image.ANTIALIAS)

            file = ImageTk.PhotoImage(file)

            self.original_img.configure(text = "", image = file)
            self.original_img.text = ""
            self.original_img.image = file
        except Exception as e:
            messagebox.showerror("An error occured !", e)
                
    def apply_filter(self):
        try:
            global cad
            print(imageNum)
            algo1 = noise_reduction("tmp/original_img.png")
            imageNumm = int(imageNum[0])
            #print(type(cad[imageNumm-1]))

            im2 = "out" + str(imageNumm) + ".png"

            path = Path('outputs/noiseRedOut')

            tempPath = os.path.join(path, im2).replace("\\","/")

            tempPath2 = os.path.join('cad',cad[imageNumm-1]).replace("\\","/")

            #print(tempPath, type(tempPath))
            algo2 = find_defects(tempPath, tempPath2)
            #print(type(algo1))

            algo1 = cv2.resize(algo1, dsize=(864,648), interpolation=cv2.INTER_CUBIC)
            algo2 = cv2.resize(algo2, dsize=(864,648), interpolation=cv2.INTER_CUBIC)

            filter_img = ImageTk.PhotoImage(image=Image.fromarray(algo2))
            self.filter_img.image = filter_img
            self.filter_img.configure(text = "", image = filter_img)
            self.filter_img.text = ""

            #print("yeyye")
        except Exception as e:
            messagebox.showerror("An error occured!", e)

        
    def show_summary(self):
        f, axarr = plt.subplots(2,2)
        axarr[0,0].imshow(Image.open(pathlib.Path("tmp/original_img.png")), cmap = "gray")
        plt.show()

        df1 = pd.read_excel('output_scores.xls', sheet_name='outputs')

        ii1 = df1['image number'].tolist()
        tt1 = df1['total time'].tolist()
        ff1 = df1['f1 score'].tolist()
        aa1 = df1['accuracy'].tolist()
        i1,t1,f1,a1 = [],[],[],[]

        for i in range(0,len(ii1)):
            i1.append([ii1[i]])
            t1.append([tt1[i]])
            f1.append([ff1[i]])
            a1.append([aa1[i]])
        
        if isinstance(imageNum, int):
            print("intt")
            im = int(imageNum)
        else :
            im = int(imageNum[0])

        return "Image " + str(i1[im]) +  "\n\tTime\t: " + str(t1[im]) + "  second" + "\nF1 score\t: " + str(f1[im]) + "\nAccuracy\t: " + str(a1[im])

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    MainApp().run()


['5']
noise_reduction for one image done!
['20']
noise_reduction for one image done!
imagenumm  19
['20']
noise_reduction for one image done!
['14']
noise_reduction for one image done!
['10']
noise_reduction for one image done!
['19']
noise_reduction for one image done!
imagenumm  18
